In [50]:
import os
import nltk
import math
import shlex
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from string import punctuation
from datetime import datetime
from nltk.util import ngrams

from pandas import Series
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer

## Processing tags

In [51]:
def remove_dig_punct(list1):
    new=[]
    for n in range (0, len(list1)):
        #removing digits
        res=''.join([i for i in list1[n] if not i.isdigit()])
        #removing punctuation
        s = ''.join(ch for ch in res if ch not in exclude)
        if (s !="") and (s not in new):
            if (len(s)>1):
                new.append(s)
    return (new)

In [4]:
data_folder_channels='/media/aruiz/data/videos_data/'
data_tags='/media/aruiz/data/videos_tags/'
community_num=[9,22,85,76,71,37,87]
lemmatizer = WordNetLemmatizer()
stopWords = set(stopwords.words('english'))
exclude = set(punctuation)

for ind in range (1,len(community_num)):
    
    # create csv to read community video tags
    file3=os.path.join(data_folder_channels,'data_community_'+str(community_num[ind])+'.csv')
    # put data into a dataframe
    df=pd.read_csv(file3)
    df=df.fillna("N/A")
    df=df[~(df['id']=='id')]
    df=df.reset_index(drop=True)
    
    # initialize lists
    all_keys_u=[] # all tag/words for each video
    all_keys_b=[] # all co-ocurrence of tags (bigrams)

    for dyear in range (2007,2017):
        total_vid=0
        
        for n in range(0,len(df)):
            date=datetime.strptime(df.publishedAt[n], '%Y-%m-%dT%H:%M:%S.%fZ')
            
            if (df.tags[n]!="N/A") and (date.year==dyear):
                total_vid+=1
                video_keys=[]

                try:
                    l = shlex.split(df.tags[n])

                except:
                    l = df.tags[n].split()

                for kw in l:
                    kw_id = kw.lower().replace("'", "").replace("´", "").replace("’", "").replace("`", "")
                    single_keys=nltk.word_tokenize(kw_id)
                    for key in single_keys:
                        key=lemmatizer.lemmatize(key)
                        if (key not in stopWords)and(len(key)>1):
                            if (key not in video_keys):
                                video_keys.append(key)

                #clean all digits and punctuation from keywords
                video_keys=remove_dig_punct(video_keys)

#                 find category of the keyword /game belongs to
                for key in video_keys:
                    all_keys_u.append(key)

                edges= itertools.combinations(video_keys_clean,2)
                for edge in edges:
                    t=(edge[0],edge[1])
                    all_keys_b.append(t)
                    
        counts_u=Series(all_keys_u)
        cu=counts_u.value_counts()
        counts_b=Series(all_keys_b)
        cb=counts_b.value_counts()

        f= open (os.path.join(data_tags,'community'+str(community_num[ind])+'_unis'+str(dyear)+'.txt'),'w')
        for j in range (0,len(cu)):
            f.write(str(cu.index[j])+" " +str(cu[j]))
            f.write('\n')
        f.write("TOTAL VIDEOS: "+str(total_vid))
        f.close()
        f= open (os.path.join(data_tags,'community'+str(community_num[ind])+'_bigs'+str(dyear)+'.txt'),'w')
        for j in range (0,len(cb)):
            f.write(str(cb.index[j][0])+" " +str(cb.index[j][1]+" "+str(cb[j])))
            f.write('\n')
        f.close()

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,9,10,11,12,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Processing categories

In [15]:
data_folder_channels='/media/aruiz/data/videos_data/'
videos_info='/media/aruiz/data/videos_info/'
community_num=[9,22,85,76,71,37,87]

for ind in range (0,len(community_num)):
# for ind in range (0,1):

    # create csv to read community video tags
    file3=os.path.join(data_folder_channels,'data_community_'+str(community_num[ind])+'.csv')
    # put data into a dataframe
    df=pd.read_csv(file3)
    df=df.fillna("N/A")
    df=df[~(df['id']=='id')]
    df=df.reset_index(drop=True)
    
    # initialize lists
    
    for dyear in range (2007,2017):
#     for dyear in range (2007,2008):
        total_vid=0
        all_categories=[] # all categories for each year (one for each video)
        
        for n in range(0,len(df)):
            date=datetime.strptime(df.publishedAt[n], '%Y-%m-%dT%H:%M:%S.%fZ')
            
            if (df.categoryId[n]!="N/A") and (date.year==dyear):
                total_vid+=1
                all_categories.append(str(df.categoryId[n]))
        
        counts_cat=Series(all_categories)
        ccat=counts_cat.value_counts()
        
        f= open (os.path.join(videos_info,'community'+str(community_num[ind])+'_categories'+'.txt'),'a')
        f.write(str(dyear)+'\n')
        for j in range (0,len(ccat)):
            f.write(str(ccat.index[j])+" " +str(ccat[j]))
            f.write('\n')
        f.write("TOTAL VIDEOS: "+str(total_vid)+'\n')
        f.close()

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,9,10,11,12,13,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,9,10,11,12,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Write csv files marking lets play videos

In [17]:
data_folder_channels='/media/aruiz/data/videos_data/'
data_islp='/media/aruiz/data/videos_data/islp/'
community_num=[9,22,85,76,71,37,87]
lemmatizer = WordNetLemmatizer()
stopWords = set(stopwords.words('english'))
exclude = set(punctuation)
t1=('let', 'play')
t2=('play', 'let')

for ind in range (0,1):

    # create csv to read community video tags
    file3=os.path.join(data_folder_channels,'data_community_'+str(community_num[ind])+'.csv')
    # put data into a dataframe
    df=pd.read_csv(file3)
    df=df.fillna("N/A")
    df=df[~(df['id']=='id')]
    df=df.reset_index(drop=True)

    for n in range(0,len(df)):
        if (df.tags[n]!="N/A"):
            date=datetime.strptime(df.publishedAt[n], '%Y-%m-%dT%H:%M:%S.%fZ')
        
            # write year and month as a new column
            df.loc[n,"year"]=date.year
            df.loc[n,"month"]=date.month
            
            #initialize container for tags
            video_keys=[]

            try:
                l = shlex.split(df.tags[n])

            except:
                l = df.tags[n].split()

            for kw in l:
                kw_id = kw.lower().replace("'", "").replace("´", "").replace("’", "").replace("`", "")
                single_keys=nltk.word_tokenize(kw_id)
                for key in single_keys:
                    key=lemmatizer.lemmatize(key)
                    if (key not in stopWords)and(len(key)>1):
                        if (key not in video_keys):
                            video_keys.append(key)

            #clean all digits and punctuation from keywords
            video_keys_clean=remove_dig_punct(video_keys)
            # create combinations of all keywords
            edges= itertools.combinations(video_keys_clean,2)

            #write a flag to identify videos tagged as letsplays
            if (t1 in edges) or (t2 in edges):
                df.loc[n,"is_letPlay"]=True
            elif ('lp' in video_keys_clean) or ('letplay' in video_keys_clean) :
                df.loc[n,"is_letPlay"]=True
            else:
                df.loc[n,"is_letPlay"]=False
        
        if n==1000:
            print ('thousand reached')
        if n==100000:
            print ('hundred thousand')
        if n==1000000:
            print ('million')
        if n==(len(df)-1):
            print ('endoffile')
            
    # write csv with islp flag column
    f=os.path.join(data_islp,'data_community_'+str(community_num[ind])+'_islp.csv')
    df.to_csv(f,encoding='utf-8', index=False)

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,9,10,11,12,13,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


thousand reached
hundred thousand
endoffile


## Read LP only videos

In [54]:
community_num=[9,22,85,76,71,37,87]
data_islp='/media/aruiz/data/videos_data/islp/'
ind=0
f=os.path.join(data_islp,'data_community_'+str(community_num[ind])+'_islp.csv')
df=pd.read_csv(f)
df=df[df['is_letPlay']==True]
df=df[df['year']==2008]
# for n in range(0,len(df)):
#         if (df.title[n]!="N/A"):
# len(df)      

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
words=df.description[16322]
s=word_grams(words,1,2)